In [1]:
import pandas as pd
import numpy as np

In [2]:
df= pd.read_excel("emailspam.xlsx")

In [3]:
df.head()

,text,spam,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109
0,Subject: naturally irresistible your corporate...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Subject: the stock trading gunslinger fanny i...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Subject: unbelievable new homes made easy im ...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Subject: 4 color printing special request add...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Subject: do not have money , get software cds ...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
columns= df.columns
columns

Index(['text', 'spam', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       ...
       'Unnamed: 100', 'Unnamed: 101', 'Unnamed: 102', 'Unnamed: 103',
       'Unnamed: 104', 'Unnamed: 105', 'Unnamed: 106', 'Unnamed: 107',
       'Unnamed: 108', 'Unnamed: 109'],
      dtype='object', length=110)

In [5]:
extra_columns = df.columns[2:110]

In [6]:
df= df.drop(columns=extra_columns)

In [7]:
df.head(3)

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1


In [8]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

In [9]:
# Load NLTK stopwords
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
stop_words

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sony\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sony\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [ ]:
df = pd.DataFrame(df)

# Function to extract features from text
def extract_features(text):
    features = []

    #Length of the string
    features.append(len(text))

    #Presence of pronouns
    pronouns = ['I', 'you', 'he', 'she', 'it', 'we', 'they']
    pronoun_presence = any(word in text.lower() for word in pronouns)
    features.append(pronoun_presence)

    #Number of pronouns
    pronoun_count = sum(1 for word in re.findall(r'\b\w+\b', text) if word.lower() in pronouns)
    features.append(pronoun_count)

    #Presence of specific symbols
    specific_symbols = ['!', '@', '#', '$', '%']
    symbol_presence = any(char in text for char in specific_symbols)
    features.append(symbol_presence)

    #Number of specific symbols
    symbol_count = sum(1 for char in text if char in specific_symbols)
    features.append(symbol_count)

    #Presence of phone numbers
    phone_number_presence = bool(re.search(r'\b\d{3}[-.\s]?\d{3}[-.\s]?\d{4}\b', text))
    features.append(phone_number_presence)

    #Presence of links
    link_presence = bool(re.search(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text))
    features.append(link_presence)

    #Presence of uppercase words
    uppercase_presence = any(word.isupper() for word in re.findall(r'\b\w+\b', text) if word.isalpha())
    features.append(uppercase_presence)

    return features

In [ ]:
df['features'] = df['text'].apply(extract_features)

In [30]:
X = pd.DataFrame(df['features'].to_list(), columns=['length', 'pronoun_presence', 'pronoun_count',
                                                    'symbol_presence', 'symbol_count', 'phone_number_presence',
                                                    'link_presence', 'uppercase_presence'])
y = df['spam']

X

,length,pronoun_presence,pronoun_count,symbol_presence,symbol_count,phone_number_presence,link_presence,uppercase_presence
0,1484,True,8,True,1,False,False,False
1,598,True,0,False,0,False,False,False
2,448,True,6,True,1,False,False,False
3,500,True,0,True,3,False,False,False
4,235,True,1,True,1,False,False,False
...,...,...,...,...,...,...,...,...
5723,1189,True,5,True,11,False,False,False
5724,1167,True,1,True,1,False,False,False
5725,2131,True,9,True,4,False,False,False
5726,1060,True,4,True,5,False,False,False


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Naive Bayes
model = MultinomialNB()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [29]:
# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

#classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.71
              precision    recall  f1-score   support

           0       0.78      0.85      0.81       856
           1       0.39      0.28      0.33       290

    accuracy                           0.71      1146
   macro avg       0.59      0.57      0.57      1146
weighted avg       0.68      0.71      0.69      1146



In [31]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

# الگوریتم Bagging: RandomForestClassifier
bagging_classifier = RandomForestClassifier(n_estimators=50, random_state=42)

# الگوریتم Boosting: AdaBoostClassifier
boosting_classifier = AdaBoostClassifier(n_estimators=50, random_state=42)

# لیست از الگوریتها
classifiers = [bagging_classifier, boosting_classifier]


classifiers_names = ['RandomForest', 'AdaBoost']

# تابعی برای انجام ارزیابی و گزارش نتایج
def evaluate_classifier(classifier, X, y):
    # ارزیابی با استفاده از 3-Fold Cross-Validation
    y_pred = cross_val_predict(classifier, X, y, cv=3)
    
    # محاسبه معیارهای ارزیابی
    precision, recall, f1, _ = precision_recall_fscore_support(y, y_pred, average='binary')
    
    cm = confusion_matrix(y, y_pred)
    
    # محاسبه خطا
    error = 1 - accuracy_score(y, y_pred)
    
    return precision, recall, f1, cm, error

# گزارش نتایج برای هر الگوریتم
for i, classifier in enumerate(classifiers):
    precision, recall, f1, cm, error = evaluate_classifier(classifier, X, y)
    
    print(f"Results for {classifiers_names[i]}:")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"Confusion Matrix:\n{cm}")
    print(f"Error: {error:.4f}")
    print("\n" + "="*50 + "\n")


Results for RandomForest:
Precision: 0.6068
Recall: 0.4861
F1-Score: 0.5398
Confusion Matrix:
[[3929  431]
 [ 703  665]]
Error: 0.1980


Results for AdaBoost:
Precision: 0.7029
Recall: 0.1937
F1-Score: 0.3037
Confusion Matrix:
[[4248  112]
 [1103  265]]
Error: 0.2121


